# サポートベクターマシン マニュアル記載例のデモ
サポートベクターマシンのマニュアル記載の例の一部を実行してみる 
https://madlib.incubator.apache.org/docs/latest/group__grp__svm.html
## 前準備

In [39]:
# ipython-sqlをロード
%reload_ext sql
# 文字コードをUTF-8にセット
# (データベースの文字コードもUTF-8にセットしておくこと)
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

## PostgreSQLに接続＆接続確認

In [40]:
%sql postgresql://postgres@centos72/postgres
%sql SELECT version();

version
"PostgreSQL 9.6.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-4), 64-bit"


## MADlibが正常にインストールされているか確認

In [41]:
%sql SELECT madlib.version();

version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-8-g82e56a4, cmake configuration time: 2016年 11月 22日 火曜日 00:25:23 UTC, build type: RelWithDebInfo, build system: Linux-3.10.0-327.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


## サンプルデータ準備

In [62]:
%%sql

DROP TABLE IF EXISTS houses;
CREATE TABLE
    houses (
     id INT
    ,tax INT
    ,bedroom INT
    ,bath FLOAT
    ,price INT
    ,size INT
    ,lot INT
    )
;
INSERT INTO
    houses
VALUES
     (1, 590, 2, 1, 50000, 770, 22100)
    ,(2, 1050, 3, 2, 85000, 1410, 12000)
    ,(3, 20, 3, 1, 22500, 1060, 3500)
    ,(4, 870, 2, 2, 90000, 1300, 17500)
    ,(5, 1320, 3, 2, 133000, 1500, 30000)
    ,(6, 1350, 2, 1, 90500, 820, 25700)
    ,(7, 2790, 3, 2.5, 260000, 2130, 25000)
    ,(8, 680, 2, 1, 142500, 1170, 22000)
    ,(9, 1840, 3, 2, 160000, 1500, 19000)
    ,(10, 3680, 4, 2, 240000, 2790, 20000)
    ,(11, 1660, 3, 1, 87000, 1030, 17500)
    ,(12, 1620, 3, 2, 118600, 1250, 20000)
    ,(13, 3100, 3, 2, 140000, 1760, 38000)
    ,(14, 2070, 2, 3, 148000, 1550, 14000)
    ,(15, 650, 3, 1.5, 65000, 1450, 12000)
;

SELECT * FROM houses;

id,tax,bedroom,bath,price,size,lot
1,590,2,1.0,50000,770,22100
2,1050,3,2.0,85000,1410,12000
3,20,3,1.0,22500,1060,3500
4,870,2,2.0,90000,1300,17500
5,1320,3,2.0,133000,1500,30000
6,1350,2,1.0,90500,820,25700
7,2790,3,2.5,260000,2130,25000
8,680,2,1.0,142500,1170,22000
9,1840,3,2.0,160000,1500,19000
10,3680,4,2.0,240000,2790,20000


## 線形モデルの分類器を作成
カーネルトリックを使わない線形モデルの作成を試みる。

In [63]:
%%sql

DROP TABLE IF EXISTS
    houses_svm
    ,houses_svm_summary
;
SELECT
    madlib.svm_classification(
         'houses'
        ,'houses_svm'
        ,'price < 100000'
        ,'ARRAY[1, tax, bath, size]'
    )
;

SELECT * FROM houses_svm;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.113989576847291, -0.00226133300755462, -0.0676303607988477, 0.00179440840634875]",9.21745071355,108.171180769,100,15,0,"[False, True]"


## 非線形モデルの分類機を作成
ガウシアンカーネルを使った非線形モデルの作成を試みる。

In [64]:
%%sql

DROP TABLE IF EXISTS
    houses_svm_gaussian
    ,houses_svm_gaussian_summary
    ,houses_svm_gaussian_random
;
SELECT
    madlib.svm_classification( 
         'houses'
        ,'houses_svm_gaussian'
        ,'price < 100000'
        ,'ARRAY[1, tax, bath, size]'
        ,'gaussian'
        ,'n_components=10'
        ,''
        ,'init_stepsize=1, max_iter=200'
    )
;

SELECT * FROM houses_svm_gaussian;

coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-2.00789985251113, 2.026255312555, -1.09903715824454, 2.04431020744862, 3.14208435637333, 0.14838741816079, 2.07527256497757, 3.0816372961301, 0.853428649404447, 3.63747384925322]",0.255909866745,0.0715415776658,184,15,0,"[False, True]"


## 線形モデル予測テスト
作成したモデルが元データに対して正しく予測できるかを試す

In [65]:
%%sql

DROP TABLE IF EXISTS houses_pred;
SELECT
    madlib.svm_predict(
         'houses_svm'
        ,'houses'
        ,'id'
        ,'houses_pred'
    )
;
SELECT
    *
    ,price < 100000 AS target
FROM
    houses
        JOIN houses_pred USING (id)
ORDER BY
    id
;

id,tax,bedroom,bath,price,size,lot,prediction,decision_function,target
1,590,2,1.0,50000,770,22100,True,0.0938672144798,True
2,1050,3,2.0,85000,1410,12000,True,0.134445050269,True
3,20,3,1.0,22500,1060,3500,True,1.90320546663,True
4,870,2,2.0,90000,1300,17500,True,0.34410006693,True
5,1320,3,2.0,133000,1500,30000,False,-0.314618105199,False
6,1350,2,1.0,90500,820,25700,False,-1.53502545094,True
7,2790,3,2.5,260000,2130,25000,False,-2.5421155107,False
8,680,2,1.0,142500,1170,22000,True,0.608110606339,False
9,1840,3,2.0,160000,1500,19000,False,-1.49051126913,False
10,3680,4,2.0,240000,2790,20000,False,-3.33657715884,False


## 非線形モデルのテスト
作成したモデルが元データに対して正しく予測できるかを試す

In [66]:
%%sql

DROP TABLE IF EXISTS houses_pred_gaussian;
SELECT
    madlib.svm_predict(
         'houses_svm_gaussian'
        , 'houses'
        , 'id'
        , 'houses_pred_gaussian'
    )
;
SELECT 
    *
    ,price < 100000 AS target
FROM
    houses
        JOIN houses_pred_gaussian USING (id)
ORDER BY
    id
;

id,tax,bedroom,bath,price,size,lot,prediction,decision_function,target
1,590,2,1.0,50000,770,22100,True,2.42762512244,True
2,1050,3,2.0,85000,1410,12000,True,1.76050726948,True
3,20,3,1.0,22500,1060,3500,True,1.14044931256,True
4,870,2,2.0,90000,1300,17500,True,1.10706143597,True
5,1320,3,2.0,133000,1500,30000,False,-1.26570595537,False
6,1350,2,1.0,90500,820,25700,True,1.33794522323,True
7,2790,3,2.5,260000,2130,25000,False,-1.0164054774,False
8,680,2,1.0,142500,1170,22000,False,-1.09640506629,False
9,1840,3,2.0,160000,1500,19000,False,-1.92695648532,False
10,3680,4,2.0,240000,2790,20000,False,-1.24283352291,False


## Area under the ROC curve による予測結果の検証

In [68]:
%%sql

DROP TABLE IF EXISTS 
     houses_pred_roc
    ,houses_pred_gaussian_roc
;
CREATE TABLE
    houses_pred_roc
AS
    SELECT 
         price < 100000 AS observed
        ,prediction
    FROM
        houses
            JOIN houses_pred USING (id)
    ORDER BY
        id
;
CREATE TABLE 
    houses_pred_gaussian_roc
AS
    SELECT 
         price < 100000 AS observed
        ,prediction
    FROM
        houses
            JOIN houses_pred_gaussian USING (id)
    ORDER BY id
;

In [69]:
%%sql

DROP TABLE IF EXISTS houses_pred_roc_output;
SELECT 
    madlib.area_under_roc(
         'houses_pred_roc'
        ,'houses_pred_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM houses_pred_roc_output;

area_under_roc
0.79464285714285714285500000000000000000000


In [71]:
%%sql

DROP TABLE IF EXISTS houses_pred_gaussian_roc_output;
SELECT
    madlib.area_under_roc(
         'houses_pred_gaussian_roc'
        ,'houses_pred_gaussian_roc_output'
        ,'prediction'
        ,'observed'
    )
;
SELECT * FROM houses_pred_gaussian_roc_output;

area_under_roc
1.00000000000000000000000000000000000000000
